In [37]:
# bot punter 

import threading
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))
from os import environ
environ['PYGAME_HIDE_SUPPORT_PROMPT'] = '1'
import pygame  # it is important to import pygame after that
import time

#Pandas Settings
pd.options.display.float_format = '{:,.2f}'.format

## Estratégias

In [ ]:
def bhct(i, lista_hct, tam_lista_hct):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')    
    # chegando se as competições do dia fazem parte da estratégia
    for i in range(tam_competicoes):
        # ligas do dia
        liga_tipster = page.find_all('table',{'class':'games'})[i].find('thead').find_all('th')[1].text
        for j in range(tam_lista_hct):
            if liga_tipster == lista_hct[j]:
                

In [ ]:
def over_25(i, lista_o25, tam_lista_o25):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')

In [ ]:
def bhft(i):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')

In [ ]:
def under_25(i):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')

In [ ]:
def back_empate(i):
    # daqui para baixo analisamos os jogos da liga que foi passada
    tam_jogos = page.find_all('table',{'class':'games'})[i].find('tbody').find_all('tr')

In [141]:
url_odds = 'https://www.oddsportal.com/soccer/italy/serie-a/parma-lecce-z7KzrgjU/'
odd = odds_under(url_odds)
print(odd)

2.22


In [99]:
def odds_home(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)

    for i in range(1, 20):  
        var = str(i)                                     
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # pegando a odd de abertura do match odds - home
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(2)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            ooh = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break
    
    driver.quit()
    
    return ooh

In [94]:
def odds_draw(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)

    # pegando a odd do match odds
    
    # draw
    for i in range(1, 20):  
        var = str(i)                                     
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # pegando a odd de abertura do match odds - draw
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(3)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            ood = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break

        
    driver.quit()
    
    return ood

In [84]:
def odds_away(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)

    # pegando a odd do match odds
    
    # away
    for i in range(1, 20):  
        var = str(i)                                     
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle': 
            
            # pegando a odd de abertura do match odds - away
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(4)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            ooa = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break
        
    driver.quit()
    
    return ooa

In [138]:
def odds_over(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)
    
    # clicando no mercado over e under
    driver.find_element_by_css_selector('.ul-nav > li:nth-child(5) > a:nth-child(1)').click()
    time.sleep(4)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    # clicando em over/under 2.5          
    driver.find_element_by_css_selector('div.table-container:nth-child(12) > div:nth-child(1) > strong:nth-child(1) > a:nth-child(1)').click()
    time.sleep(1)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    
    # odd over 2.5
    for i in range(1, 20):  
        var = str(i)                          
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # odd de abertura de over 2.5               
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(3)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            opening_odd_over = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break

    
    driver.quit()
    
    return opening_odd_over

In [140]:
def odds_under(url):
    # descomentar estas 4 próximas linhas para usar o navegador invisível
    gecko = '/home/andre/Downloads/geckodriver'
    driver = webdriver.FirefoxOptions()
    driver.add_argument('-headless')
    driver = webdriver.Firefox(executable_path=gecko, options=driver)

    # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
    #driver = webdriver.Firefox() 
    driver.get(url)
    time.sleep(1)
    
    # clicando no mercado over e under
    driver.find_element_by_css_selector('.ul-nav > li:nth-child(5) > a:nth-child(1)').click()
    time.sleep(4)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    # clicando em over/under 2.5          
    driver.find_element_by_css_selector('div.table-container:nth-child(12) > div:nth-child(1) > strong:nth-child(1) > a:nth-child(1)').click()
    time.sleep(1)
    
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    
    # odd under 2.5
    for i in range(1, 20):  
        var = str(i)                          
        bookmaker = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(1) > div:nth-child(1) > a:nth-child(2)').text
        if bookmaker == 'Pinnacle':     
            
            # odd de abertura de under 2.5               
            elem_opening_odd = driver.find_element_by_css_selector('tr.lo:nth-child('+ var +') > td:nth-child(4)')
            hov = ActionChains(driver).move_to_element(elem_opening_odd)
            hov.perform()
            data_in_the_bubble = driver.find_element_by_xpath("//*[@id='tooltiptext']")
            opening_odd_under = data_in_the_bubble.get_attribute("innerHTML").split('Opening odds:')[1].split('<strong>')[1].split('</strong>')[0]
            break

    
    driver.quit()
    
    return opening_odd_under

In [ ]:
# Script principal

# passar todo dia para o bot o link com a data futura de jogos para ele analisar
data = '12-01-2020'
url = 'https://tipsterarea.com/matches/date-' + data'      
link = requests.get(url)
page = BeautifulSoup(link.text, 'html.parser')

# lista de todas as competições do betaminic do jeito que ta escrito no site tipster
leagues = []
tam_leagues = len(leagues)

# lista de ligas do back home contra tendencia
lista_hct = []
tam_lista_hct = len(lista_hct)

# lista de ligas do over 2.5 
lista_o25 = []
tam_lista_o25 = len(lista_o25)

# competições do site tipster
tam_competicoes = len(page.find_all('table',{'class':'games'}))            

# chegando se as competições do dia fazem parte das do betaminic
for i in range(tam_competicoes):
    # ligas do dia
    liga_tipster = page.find_all('table',{'class':'games'})[i].find('thead').find_all('th')[1].text

    for j in range(tam_leagues):
        if liga_tipster == leagues[j]:
            # verificando se os filtros das estratégias são atendidos
            bhct(i, lista_hct, tam_lista_hct)
            over_25(i, lista_o25, tam_lista_o25)
            bhft(i)
            under_25(i)
            back_empate(i)
            break